<a href="https://colab.research.google.com/github/michellamello/mackenzie/blob/main/plpcd6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importa bibliotecas necessárias

In [ ]:
library(tidyverse) # Collection of all the good stuff like dplyr, ggplot2 ect.
library(devtools)
library(dplyr)

#Realiza a leitura da fonte de dados CSV

In [ ]:
voos2008 = read.csv("2008.csv")

head(voos2008)

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,⋯,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<int>,⋯,<int>,<int>,<int>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>
1,2008,1,3,4,1343,1325,1451,1435,WN,588,⋯,4,9,0,,0,16,0,0,0,0
2,2008,1,3,4,1125,1120,1247,1245,WN,1343,⋯,3,8,0,,0,NA,NA,NA,NA,NA
3,2008,1,3,4,2009,2015,2136,2140,WN,3841,⋯,2,14,0,,0,NA,NA,NA,NA,NA
4,2008,1,3,4,903,855,1203,1205,WN,3,⋯,5,7,0,,0,NA,NA,NA,NA,NA
5,2008,1,3,4,1423,1400,1726,1710,WN,25,⋯,6,10,0,,0,16,0,0,0,0
6,2008,1,3,4,2024,2020,2325,2325,WN,51,⋯,13,7,0,,0,NA,NA,NA,NA,NA


#Seleciona apenas colunas necessárias para a apresentação do resultado

In [ ]:
novoDataset = voos2008 %>%
select(Year,Month,DayofMonth,DepTime,DepDelay,ArrDelay,Cancelled)

str(novoDataset)
head(novoDataset)

'data.frame':	1906825 obs. of  7 variables:
 $ Year      : int  2008 2008 2008 2008 2008 2008 2008 2008 2008 2008 ...
 $ Month     : int  1 1 1 1 1 1 1 1 1 1 ...
 $ DayofMonth: int  3 3 3 3 3 3 3 3 3 3 ...
 $ DepTime   : int  1343 1125 2009 903 1423 2024 1753 622 1944 1453 ...
 $ DepDelay  : int  18 5 -6 8 23 4 8 2 -1 28 ...
 $ ArrDelay  : int  16 2 -4 -2 16 0 3 5 -5 26 ...
 $ Cancelled : int  0 0 0 0 0 0 0 0 0 0 ...


,Year,Month,DayofMonth,DepTime,DepDelay,ArrDelay,Cancelled
,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,2008,1,3,1343,18,16,0
2,2008,1,3,1125,5,2,0
3,2008,1,3,2009,-6,-4,0
4,2008,1,3,903,8,-2,0
5,2008,1,3,1423,23,16,0
6,2008,1,3,2024,4,0,0


#Agrupa os atrasos por mês, cria uma variável atrasoMedio e calcula a média mensal de atrasos para verificar o mês com menor quantidade de atrasos

In [ ]:
novoDataset %>%
select(Month, DepDelay) %>%
group_by(Month) %>%
mutate(atrasoMedio = mean(DepDelay, na.rm = TRUE)) %>%
select(Month, atrasoMedio) %>%
unique() %>%

arrange(atrasoMedio)

Month,atrasoMedio
<int>,<dbl>
4,7.78768
1,11.47610
3,12.49127
2,13.70623


#Seleciona apenas origens e atrasos do dataframe original

In [ ]:
voos2008 %>%
select(Origin, DepDelay) %>%
head(10)

,Origin,DepDelay
,<chr>,<int>
1,HOU,18
2,HOU,5
3,HOU,-6
4,HOU,8
5,HOU,23
6,HOU,4
7,HOU,8
8,HOU,2
9,HOU,-1


#Com o objetivo de responder qual o aeroporto onde existem mais atrasos, seleciona apenas os códigos dos aeroportos e atrasos, agrupa por aeroportos, soma atrasos, mantém apenas valores únicos, ordena de forma decrescente por atraso e exibe os 10 primeiros resultados

In [ ]:
voos2008 %>%
select(Origin,DepDelay) %>%
group_by(Origin) %>%
mutate(totalAtraso = sum(DepDelay, na.rm = TRUE)) %>%
select(Origin,totalAtraso) %>%
unique()%>%
arrange(desc(totalAtraso)) %>%
head(10)

Origin,totalAtraso
<chr>,<int>
ORD,2048013
ATL,1365441
DFW,926051
EWR,700200
DEN,660502
LAX,648098
IAH,629610
LAS,623521
SFO,621959


#Filtra voos cancelados, os agrupa e soma por motivo do cancelamento

In [ ]:
voosCancelados = voos2008 %>%
select(Cancelled, CancellationCode) %>%
filter(Cancelled == 1) %>%
group_by(CancellationCode) %>%
mutate(totalAtraso = sum(Cancelled)) %>%
select(CancellationCode, totalAtraso) %>%
unique() %>%
arrange(desc(totalAtraso))

head(voosCancelados)

CancellationCode,totalAtraso
<chr>,<int>
B,24298
A,19235
C,11181
D,6


#Cria dataframe associando códigos de cancelamento aos motivos

In [ ]:
codigoCancelamento = c("A", "B", "C", "D")
descricaoCancelamento = c("Operadora", "Clima", "NAS", "Segurança")

tiposOcorrencias = data.frame(Codigo = codigoCancelamento, Descricao = descricaoCancelamento)

head(tiposOcorrencias)

,Codigo,Descricao
,<chr>,<chr>
1,A,Operadora
2,B,Clima
3,C,NAS
4,D,Segurança


#Realiza inner join entre dataframe de códigos de cancelamento e novo dataframe criado com os motivos de cancelamento, usando como chave o código

In [ ]:
inner_join(x = voosCancelados,y = tiposOcorrencias, by = c("CancellationCode" = "Codigo"))

CancellationCode,totalAtraso,Descricao
<chr>,<int>,<chr>
B,24298,Clima
A,19235,Operadora
C,11181,NAS
D,6,Segurança
